# Digital epidemiology
### University of Trento
### AA 2024/2025

Author: Michele Tizzoni

---
## Notebook 4
### Epidemics on networks

In this notebook, we'll use the Python library ["Epidemics on Networks" developed by Kiss, Miller & Simon](https://github.com/springer-math/Mathematics-of-Epidemics-on-Networks).

The library must be installed using pip:

    pip install EoN
  

The library documentation is [available here](http://epidemicsonnetworks.readthedocs.io/en/latest/). 

In [ ]:
import EoN
import networkx as nx
import numpy as np
from collections import defaultdict
import pandas as pd
import seaborn as sns
import matplotlib.ticker as ticker
from operator import itemgetter


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# Simulate the spread of a SIR epidemic on the US airport network

In [ ]:
airport_path = "./../datasets/USairport_2010.txt"
meta_path = "./../datasets/USairport_2010_codes.txt"

In [ ]:
G = nx.Graph()
fh = open(airport_path, "r")
for line in fh.readlines():
    s = line.strip().split()
    G.add_edge(int(s[0]), int(s[1]))
fh.close()

In [ ]:
G.code = {}
G.name = {}
G.pos = {}

lons = []
lats = []

finfo = open(meta_path, "r")
for line in finfo.readlines():
    s = line.strip().split()
    node = int(s[0])

    lon = float(s[4])
    lat = float(s[3])

    G.code[node] = s[1]
    G.name[node] = s[2]
    G.pos[node] = [lon, lat]

    lons.append(lon)
    lats.append(lat)
finfo.close()

## SIR model definition

In [ ]:
# disease parameters
mu = 0.1  # infectious period
lambd = 0.01  # probability of infection given a contact

In [ ]:
# we need to store the disease status of each node
G.disease_status = {}  # S=0, I=1, R=-1

infected_nodes = []  # list of infected nodes

In [ ]:
# let's choose a seed
node_list = []

deg = dict(G.degree())
for i in sorted(deg.items(), key=itemgetter(1)):
    node_list.append(i[0])
seed = node_list[-1]

print("The seed is", G.name[seed])
print("The degree of the seed is", G.degree(seed))

In [ ]:
# initialize the network
infected_nodes.append(seed)

for n in G.nodes():
    if n in infected_nodes:
        G.disease_status[n] = 1
        # infected
    else:
        G.disease_status[n] = 0
        # susceptible

## SIR simulation

In [ ]:
I_net = []

while len(infected_nodes) > 0:

    # transmission
    for i in infected_nodes:
        for j in G.neighbors(i):
            if G.disease_status[j] == 0:
                p = np.random.random()
                if p < lambd:
                    G.disease_status[j] = 1

    # recovery
    for k in infected_nodes:
        p = np.random.random()
        if p < mu:
            G.disease_status[k] = -1

    # update of disease status
    infected_nodes = []
    for n in G.nodes():
        if G.disease_status[n] == 1:
            infected_nodes.append(n)

    # store output
    I_net.append(len(infected_nodes))

In [ ]:
plt.figure(figsize=(8, 6))

plt.xlabel("time", fontsize=18)
plt.ylabel("prevalence", fontsize=18)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.plot(range(0, len(I_net)), I_net)

In [ ]:
recovered = 0
for n in G.nodes():
    if G.disease_status[n] == -1:
        recovered += 1

print("The total number of infected nodes is", recovered)
print("The final attack rate is", recovered / len(G.nodes()))

## Combine NetworkX and Geopandas to visualize the spreading

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
from pyproj import Transformer, transform

In [ ]:
shape_path = "./../shapefiles/USA_shape.shp"

We import the USA shapefile

In [ ]:
usa = gpd.read_file(shape_path)

In [ ]:
usa.plot(figsize=(12, 7))

In [ ]:
usa.head()

In [ ]:
usa_cont = usa[
    (usa.NAME != "Alaska") & (usa.NAME != "Hawaii") & (usa.NAME != "Puerto Rico")
]

In [ ]:
usa_cont.head()

In [ ]:
usa_cont.plot(figsize=(12, 7))

In [ ]:
usa_cont.crs

In [ ]:
usa_cont_alb = usa_cont.to_crs('esri:102003')

In [ ]:
ax = usa_cont_alb.plot(figsize=(10, 5), alpha=0.8)

In [ ]:
usa_cont_alb.crs

Load metadata

In [ ]:
df = pd.read_csv(meta_path, sep=" ", names=["id", "code", "name", "lat", "lon"])

In [ ]:
df

In [ ]:
geo = [Point(xy) for xy in zip(df.lon, df.lat)]
#crs = {"init": "epsg:4326"}
crs=4326
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geo)

In [ ]:
geo_df.crs

In [ ]:
geo_df.plot(figsize=(10, 7))

In [ ]:
original = 'epsg:4326'  # EPSG:4326 in your case
destination =   usa_cont_alb.crs # your new proj

In [ ]:
transformer = Transformer.from_crs(original, destination)


In [ ]:
G.pos_new = {}

for node in G:

    long, lat = G.pos[node]
    
    x, y = transformer.transform(lat, long)

    G.pos_new[node] = (x, y)

In [ ]:
ax = usa_cont_alb.plot(figsize=(12, 7), alpha=1.)

nx.draw_networkx_nodes(
    G,
    pos=G.pos_new,
    node_size=30,
    node_color='black'
)

# SIR simulation

In [ ]:
G.disease_status = {}  # S=0, I=1, R=-1

infected_nodes = []  # list of infected nodes

In [ ]:
# initialize the network
infected_nodes.append(seed)

for n in G.nodes():
    if n in infected_nodes:
        G.disease_status[n] = 1
        # infected
    else:
        G.disease_status[n] = 0
        # susceptible

In [ ]:
t = 0
node_color = [G.disease_status[v] for v in G]  # color code on disease status

In [ ]:
inf_G = nx.DiGraph()

while len(infected_nodes) > 0 and t < 15:
    
    for i in infected_nodes:
        for j in G.neighbors(i):
            if G.disease_status[j] == 0:
                p = np.random.random()
                if p < lambd:
                    G.disease_status[j] = 1
                    inf_G.add_edge(i,j)

    for k in infected_nodes:
        p = np.random.random()
        if p < mu:
            G.disease_status[k] = -1

    infected_nodes = []
    for n in G.nodes():
        if G.disease_status[n] == 1:
            infected_nodes.append(n)

    t += 1
    node_color = [G.disease_status[v] for v in G]  # color code on disease status

    plt.figure(figsize=(12, 7))
    
    ax = usa_cont_alb.plot(figsize=(12, 7), alpha=0.7)
    
    nx.draw_networkx_nodes(
        G,
        pos=G.pos_new,
        node_size=30,
        node_color=node_color,
        cmap=plt.cm.RdBu_r,
        vmin=-1,
        vmax=1,
    )
    
    nx.draw_networkx_edges(
        inf_G,
        pos=G.pos_new,
    )
    
    inf_G.clear_edges()



---
# Exploring the epidemic threshold for different network topologies

## Epidemic threshold for homogeneous networks

We simulate the spread of an SIR on an Erdos-Renyi graph with constant recovery rate.

In [ ]:
N=10000
p=0.002
G=nx.fast_gnp_random_graph(N, p)

In [ ]:
nx.is_connected(G)

In [ ]:
print(len(G))
print(len(G.edges))

---
#### For this network the epidemic threshold can be approximated as 

$\lambda_c = \frac{\mu}{\langle k \rangle}$

In [ ]:
mu=0.2

In [ ]:
avg_deg1=2*len(G.edges)/N
lc=mu/avg_deg1
print(lc)

**As expected for this network, we have $\langle k^2 \rangle \sim \langle k \rangle^2 + \langle k \rangle$**

In [ ]:
sum_k2=0
for i in G.nodes():
    
    k=G.degree(i)
    sum_k2+=k*k

avg_k2=sum_k2/N
print(avg_k2)

In [ ]:
avg_deg1**2 + avg_deg1

### Simulations of an SIR process
We simulate 20 realizations of a SIR model for increasing values of $\lambda$ using the [fast_SIR function of EoN](https://epidemicsonnetworks.readthedocs.io/en/latest/functions/EoN.fast_SIR.html?highlight=fast_SIR)

In [ ]:
final_size=defaultdict(list)

for lambd in np.geomspace(0.0001, 1.0, 20):
    
    for r in range(0, 20):
        initial_size = 500
        
        t, S, I, R = EoN.fast_SIR(G, lambd, mu, initial_infecteds = range(initial_size))
        
        final_size[lambd].append(R[-1]/N)

In [ ]:
homo_net_size=pd.DataFrame.from_dict(final_size)

In [ ]:
homo_net_size

In [ ]:
plt.figure(figsize=(12,7))

homo_net_size.boxplot(positions=np.array(homo_net_size.columns), 
                      widths=np.array(homo_net_size.columns)/3)

plt.vlines(x=lc, ymin=0.045, ymax=1.1)

plt.xscale('log')
plt.yscale('log')
plt.xlim(0.0001, 1.0)
plt.ylim(0.045, 1.1)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel('final epidemic size', fontsize=18)
plt.xlabel('$\lambda$', fontsize=18)

## Epidemic threshold for Barabàsi-Albert model networks

In [ ]:
N=10000
AB=nx.barabasi_albert_graph(N, 10)

In [ ]:
nx.is_connected(AB)

In [ ]:
print(len(AB.edges()))

In [ ]:
sum_k2=0
for i in AB.nodes():
    k=AB.degree(i)
    sum_k2+=k*k
avg_k2=sum_k2/N
print(avg_k2)    

In [ ]:
avg_deg=2*len(AB.edges)/N
print(avg_deg)

**The threshold can be approximated as $\lambda_c \sim \mu \frac{\langle k \rangle}{\langle k^2 \rangle - \langle k \rangle}$**

In [ ]:
lambda_c=mu*avg_deg/(avg_k2-avg_deg)
print(lambda_c)

In [ ]:
lambda_c/lc

### Simulations of an SIR process
We simulate 20 realizations of a SIR model for increasing values of $\lambda$

In [ ]:
final_size_AB=defaultdict(list)

for lambd in np.geomspace(0.0001, 1.0, 20):
    for r in range(0, 20):
        initial_size = 500
        t, S, I, R = EoN.fast_SIR(AB, lambd, mu, initial_infecteds = range(initial_size))
        
        final_size_AB[lambd].append(R[-1]/N)

In [ ]:
sf_net_size=pd.DataFrame.from_dict(final_size_AB)

In [ ]:
sf_net_size.tail()

In [ ]:
plt.figure(figsize=(12,7))

homo_net_size.boxplot(positions=np.array(homo_net_size.columns), widths=np.array(homo_net_size.columns)/3, color='r' )

plt.vlines(x=lambda_c, ymin=0.04, ymax=1.1)
sf_net_size.boxplot(positions=np.array(sf_net_size.columns), widths=np.array(sf_net_size.columns)/3)

plt.yscale('log')
plt.xscale('log')
plt.xlim(0.0001, 1.0)
plt.ylim(0.045, 1.1)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel('final epidemic size', fontsize=18)
plt.xlabel('$\lambda$', fontsize=18)